In [1]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! pip install transformers[sentencepiece] datasets
! pip install sacrebleu sentencepiece
! pip install huggingface_hub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install wandb -q

In [4]:
# Importing libraries
import collections
import helper
import numpy as np
import tensorflow as tf
import pandas as pd

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer

from transformers import create_optimizer, AdamWeightDecay
from transformers import DataCollatorForSeq2Seq
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

from datasets.dataset_dict import DatasetDict
from datasets import Dataset
from keras.preprocessing.text import Tokenizer


In [5]:
# I have cleaned the data from my jupyter notebook
df = pd.read_csv('/content/drive/MyDrive/Kinya_English_sentences.csv')
df

,Kinyarwanda,English
0,Ababyeyi babo bamaze gupfa basogokuru barabare...,After their parents died their grandparents ra...
1,Waba uzi umubare wabantu bitabiriye kubyina ic...,Do you know the number of people who attended ...
2,Jya kuryama Ugomba kubyuka kare ejo mugitondo.,Go to bed You have to wake up early tomorrow m...
3,Nzasimbuza igihe cyatakaye nkora uko nshoboye.,I will replace the lost time by doing my best.
4,Ngomba kumanika nonaha. Umuntu ategereje gukor...,I have to hang up now. Someone is waiting to u...
...,...,...
25008,Ntabwo nkunda igitabo gishya dukoresha murwego...,I don't i like book new we use at the level of...
25009,Sinshaka kurya muri iyo resitora. Ibiryo birab...,I don't want to to eat in salt restaurant . Fo...
25010,Ntabwo ntekereza ko nzanyura muriyi mirimo yos...,I don't i think no i will go through in this j...
25011,Ntabwo ntekereza ko iyi modoka ishaje izagera ...,I don't i think no this one car old will arriv...


In [6]:
Kinyarwanda = list(df["Kinyarwanda"])

English = list(df["English"])

In [7]:
print(len(Kinyarwanda))
print(len(English))

25013
25013


In [8]:
Kinyarwanda[7630]

'umuti uvura umutwe'

In [9]:
English[7630]


'the cure medecine of a head'

In [10]:
for sample_i in range(2):
    print('kinyarwanda Line {}:  {}'.format(sample_i + 1, Kinyarwanda[sample_i]))
    print('english Line {}:  {}'.format(sample_i + 1, English[sample_i]))

kinyarwanda Line 1:  Ababyeyi babo bamaze gupfa basogokuru barabareze. 
english Line 1:  After their parents died their grandparents raised them.
kinyarwanda Line 2:  Waba uzi umubare wabantu bitabiriye kubyina icyumweru gishize? 
english Line 2:  Do you know the number of people who attended the dance last week?


In [11]:
kinya_words_counter = collections.Counter([word for sentence in Kinyarwanda for word in sentence.split()])
eng_words_counter = collections.Counter([word for sentence in English for word in sentence.split()])

print('{} Kinyarwanda words.'.format(len([word for sentence in Kinyarwanda for word in sentence.split()])))
print('{} unique Kinyarwanda words.'.format(len(kinya_words_counter)))
print('10 Most common words in the Kinyarwanda sets:')
print('"' + '" "'.join(list(zip(*kinya_words_counter.most_common(10)))[0]) + '"')
print()

print('{} English words.'.format(len([word for sentence in English for word in sentence.split()])))
print('{} unique english words.'.format(len(eng_words_counter)))
print('10 Most common words in the english dataset:')
print('"' + '" "'.join(list(zip(*eng_words_counter.most_common(10)))[0]) + '"')

148619 Kinyarwanda words.
32327 unique Kinyarwanda words.
10 Most common words in the Kinyarwanda sets:
"ko" "mu" "ku" "na" "Ntabwo" "muri" "ikintu" "hari" "icyo" "gukora"

208822 English words.
14352 unique english words.
10 Most common words in the english dataset:
"the" "to" "I" "you" "of" "a" "in" "is" "and" "he"


In [12]:
Kinyarwanda[:3]

['Ababyeyi babo bamaze gupfa basogokuru barabareze. ',
 'Waba uzi umubare wabantu bitabiriye kubyina icyumweru gishize? ',
 'Jya kuryama Ugomba kubyuka kare ejo mugitondo. ']

In [13]:
English[:3]

['After their parents died their grandparents raised them.',
 'Do you know the number of people who attended the dance last week?',
 'Go to bed You have to wake up early tomorrow morning.']

In [14]:
translation=[]
for k,j in zip(Kinyarwanda,English):
  translation.append({"kiny":k,"en":j})

translation

train = {"id":np.array(range(len(translation)),dtype=np.str),"translation":translation}
# train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


In [15]:
d = {'train':Dataset.from_dict(train),
     }

data_language = DatasetDict(d)
data_language


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 25013
    })
})

In [16]:
data_language["train"]["translation"][2]

{'en': 'Go to bed You have to wake up early tomorrow morning.',
 'kiny': 'Jya kuryama Ugomba kubyuka kare ejo mugitondo. '}

OURS

USING TENSORFLOW


In [17]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [18]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base") 

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [20]:
data_language = data_language["train"].train_test_split(test_size=0.2)
data_language

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 20010
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 5003
    })
})

In [21]:
source_lang = "kiny"

target_lang = "en"

prefix = "translate Kinyarwanda to English: "


def preprocess_function(examples):

    inputs = [prefix + example[source_lang] for example in examples["translation"]]

    targets = [example[target_lang] for example in examples["translation"]]

    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)

    return model_inputs

In [22]:
tokenized_words = data_language.map(preprocess_function, batched=True)

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [23]:
tokenized_words

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20010
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5003
    })
})

In [24]:
tf_train_set = model.prepare_tf_dataset(

    tokenized_words["train"],

    shuffle=True,

    batch_size=4,

    collate_fn=data_collator,

)

tf_test_set = model.prepare_tf_dataset(

    tokenized_words["test"],

    shuffle=False,

    batch_size=4,

    collate_fn=data_collator,

)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import wandb
wandb.login(key="4e8aa8d9bd8a1c30f3c3ba05019dccea2fad47b8")

In [ ]:
run = wandb.init(
    name = "submission", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "Kiny-Eng-Translation", ### Project should be created in your wandb account 
    # config = config ### Wandb Config for your run
)

In [25]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [26]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [27]:
model.fit(tf_train_set, validation_data=tf_test_set, epochs=20)

Epoch 1/20
5002/5002 [==============================] - 1087s 210ms/step - loss: 3.0205 - val_loss: 2.5312
Epoch 2/20
5002/5002 [==============================] - 1036s 207ms/step - loss: 2.4663 - val_loss: 2.1170
Epoch 3/20
5002/5002 [==============================] - 1028s 205ms/step - loss: 2.1218 - val_loss: 1.8931
Epoch 4/20
5002/5002 [==============================] - 1019s 204ms/step - loss: 1.8769 - val_loss: 1.7303
Epoch 5/20
5002/5002 [==============================] - 1013s 202ms/step - loss: 1.6931 - val_loss: 1.6240
Epoch 6/20
5002/5002 [==============================] - 1003s 201ms/step - loss: 1.5406 - val_loss: 1.5399
Epoch 7/20
5002/5002 [==============================] - 997s 199ms/step - loss: 1.4169 - val_loss: 1.4766
Epoch 8/20
5002/5002 [==============================] - 995s 199ms/step - loss: 1.3107 - val_loss: 1.4238
Epoch 9/20
5002/5002 [==============================] - 1017s 203ms/step - loss: 1.2157 - val_loss: 1.3951
Epoch 10/20
5002/5002 [================

In [2]:
# input_text  = "Waba uzi umubare wabantu bitabiriye kubyina icyumweru gishize "
input_text = "ntabwo ari byiza guhora wicuza ibyo wakoze utabizi"
# if 't5' in model: 
input_text = "translate Kinyarwanda to English: " + input_text
tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

In [39]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

It is not good to always regret what you have done for yourself
